# Team description

Team: Ehhhhhhh

Markus Kinn, 106660
Mario Haroun, 543915
Torstein Korten, 543955

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer, IterativeImputer
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_absolute_error
import numpy as np
import itertools

from sklearn.model_selection import train_test_split


def remove_unwanted_rows(df):
    unwanted_rows = (df['direct_rad:W'] == 0) & (df['diffuse_rad:W'] == 0) & (df['pv_measurement'] > 200) & (df['sun_elevation:d'] < 0) & (df['is_day:idx'] == 0)
    cleaned_df = df[~unwanted_rows]
    return cleaned_df
def remove_highly_correlated_features(df, threshold):
    # Compute the Pearson correlation matrix
    correlation_matrix = df.corr(method='pearson')

    # Initialize an empty list to hold features to be removed
    features_to_remove = []

    # Traverse the correlation matrix to find highly correlated features
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            feature1 = correlation_matrix.columns[i]
            feature2 = correlation_matrix.columns[j]

            # Check for high absolute correlation
            if abs(correlation_matrix.iloc[i, j]) > threshold:
                # Add one of the features to the list if it's not already there
                if feature1 not in features_to_remove and feature2 not in features_to_remove:
                    features_to_remove.append(feature1)

    # Drop the identified features from the DataFrame
    filtered_df = df.drop(columns=features_to_remove)

    return filtered_df

def find_long_constant_periods(data, threshold):
    start = None
    segments = []
    for i in range(1, len(data)):
        if data[i] == data[i-1] and data[i] != 0:
            if start is None:
                start = i-1
        else:
            if start is not None:
                if (i - start) > threshold:
                    segments.append((start, i))
                start = None
    return segments

def remove_constant_periods(df, segments):
    drop_indices = []
    for start, end in segments:
        drop_indices.extend(range(start, end))
    return df.drop(drop_indices)

def lag_features_by_one_hour(df, column_names, time_col='time'):

    # Check if the DataFrame has a time-based index
    df['index'] = df[time_col]
    df = df.set_index('index')

    # Loop through each column name to create a lagged feature
    for col in column_names:
        lagged_col_name = f"{col}"
        df[lagged_col_name] = df[col].shift(freq='-1H')

    return df

def is_estimated(df, time_col='time'):
    split_date = '2022-10-27'
    df['is_estimated'] = 0  # Initialize with 0 (indicating observed)
    df.loc[df[time_col] >= pd.Timestamp(split_date), 'is_estimated'] = 1  # Set 1 for estimated data
    return df

def resample_to_hourly(df, datetime_column='date_forecast'):
    df[datetime_column] = pd.to_datetime(df[datetime_column])
    df.sort_values(by=datetime_column, inplace=True)

    df.set_index(datetime_column, inplace=True)

    df_hourly = df.resample('H').mean()

    df_hourly.dropna(how='all', inplace=True)

    df_hourly.reset_index(inplace=True)

    return df_hourly

def generate_solar_features_1(data):
    relevant_features = [
        'direct_rad:W', 'clear_sky_rad:W', 'diffuse_rad:W', 'sun_elevation:d', 'sun_azimuth:d',
        'clear_sky_energy_1h:J', 'direct_rad_1h:J', 'effective_cloud_cover:p', 'diffuse_rad_1h:J',
        'is_in_shadow:idx', 'total_cloud_cover:p', 'wind_speed_u_10m:ms', 'snow_water:kgm2',
        'relative_humidity_1000hPa:p', 'is_day:idx', 'wind_speed_v_10m:ms', 'cloud_base_agl:m',
        'fresh_snow_24h:cm', 'wind_speed_10m:ms', 'pressure_100m:hPa'
    ]

    interactions = {}
    ratios = {}
    differences = {}
    lags = {}
    self_interactions = {}
    additive = {}

    for col_pair in itertools.combinations(relevant_features, 2):
        interactions[f'{col_pair[0]}_times_{col_pair[1]}'] = data[col_pair[0]] * data[col_pair[1]]
        ratios[f'{col_pair[0]}_div_{col_pair[1]}'] = data[col_pair[0]] / (data[col_pair[1]] + 1e-8)
        differences[f'{col_pair[0]}_minus_{col_pair[1]}'] = data[col_pair[0]] - data[col_pair[1]]
        additive[f'{col_pair[0]}_plus_{col_pair[1]}'] = data[col_pair[0]] + data[col_pair[1]]

    for col in relevant_features:
        self_interactions[f'{col}_squared'] = data[col] ** 2

    # Creating lags for all relevant features
    for col in relevant_features:
        lags[f'{col}_lag1'] = data[col].shift(1)
        lags[f'{col}_lag3'] = data[col].shift(3)

    # Concatenate all new features with the original data
    data = pd.concat([data, pd.DataFrame(interactions), pd.DataFrame(ratios),
                      pd.DataFrame(differences), pd.DataFrame(lags), pd.DataFrame(self_interactions), pd.DataFrame(additive)], axis=1)

    data['wind_magnitude'] = np.sqrt(data['wind_speed_u_10m:ms']**2 + data['wind_speed_v_10m:ms']**2)
    data['wind_direction'] = np.arctan2(data['wind_speed_v_10m:ms'], data['wind_speed_u_10m:ms'])
    data['solar_angle_impact'] = np.sin(np.radians(data['sun_elevation:d']))

    return data

def generate_solar_features_2(data):
    relevant_features = [
        'sun_elevation:d', 'clear_sky_rad:W', 'direct_rad:W', 'diffuse_rad:W',
        'sun_azimuth:d', 'clear_sky_energy_1h:J', 'cloud_base_agl:m', 'diffuse_rad_1h:J',
        'effective_cloud_cover:p', 'direct_rad_1h:J', 'snow_water:kgm2', 'is_in_shadow:idx',
        'fresh_snow_24h:cm', 'wind_speed_u_10m:ms', 'total_cloud_cover:p', 'msl_pressure:hPa',
        'is_day:idx', 'relative_humidity_1000hPa:p', 'pressure_100m:hPa', 'ceiling_height_agl:m'
    ]

    interactions = {}
    ratios = {}
    differences = {}
    lags = {}
    self_interactions = {}
    additive = {}

    for col_pair in itertools.combinations(relevant_features, 2):
        interactions[f'{col_pair[0]}_times_{col_pair[1]}'] = data[col_pair[0]] * data[col_pair[1]]
        ratios[f'{col_pair[0]}_div_{col_pair[1]}'] = data[col_pair[0]] / (data[col_pair[1]] + 1e-8)
        differences[f'{col_pair[0]}_minus_{col_pair[1]}'] = data[col_pair[0]] - data[col_pair[1]]
        additive[f'{col_pair[0]}_plus_{col_pair[1]}'] = data[col_pair[0]] + data[col_pair[1]]

    for col in relevant_features:
        self_interactions[f'{col}_squared'] = data[col] ** 2

    # Creating lags for all relevant features
    for col in relevant_features:
        lags[f'{col}_lag1'] = data[col].shift(1)
        lags[f'{col}_lag3'] = data[col].shift(3)

    # Concatenate all new features with the original data
    data = pd.concat([data, pd.DataFrame(interactions), pd.DataFrame(ratios),
                      pd.DataFrame(differences), pd.DataFrame(lags), pd.DataFrame(self_interactions), pd.DataFrame(additive)], axis=1)

    data['wind_magnitude'] = np.sqrt(data['wind_speed_u_10m:ms']**2 + data['wind_speed_v_10m:ms']**2)
    data['wind_direction'] = np.arctan2(data['wind_speed_v_10m:ms'], data['wind_speed_u_10m:ms'])
    data['solar_angle_impact'] = np.sin(np.radians(data['sun_elevation:d']))

    return data

def generate_solar_features_3(data):
    relevant_features = [
        'clear_sky_rad:W', 'clear_sky_energy_1h:J', 'direct_rad_1h:J', 'sun_elevation:d',
        'direct_rad:W', 'diffuse_rad:W', 'sun_azimuth:d', 'diffuse_rad_1h:J',
        'air_density_2m:kgm3', 'wind_speed_v_10m:ms', 'fresh_snow_24h:cm',
        'relative_humidity_1000hPa:p', 'total_cloud_cover:p', 'effective_cloud_cover:p',
        'cloud_base_agl:m', 'snow_water:kgm2', 't_1000hPa:K', 'is_in_shadow:idx', 'dew_point_2m:K',
        'pressure_100m:hPa'
    ]

    interactions = {}
    ratios = {}
    differences = {}
    lags = {}
    self_interactions = {}
    additive = {}

    for col_pair in itertools.combinations(relevant_features, 2):
        interactions[f'{col_pair[0]}_times_{col_pair[1]}'] = data[col_pair[0]] * data[col_pair[1]]
        ratios[f'{col_pair[0]}_div_{col_pair[1]}'] = data[col_pair[0]] / (data[col_pair[1]] + 1e-8)
        differences[f'{col_pair[0]}_minus_{col_pair[1]}'] = data[col_pair[0]] - data[col_pair[1]]
        additive[f'{col_pair[0]}_plus_{col_pair[1]}'] = data[col_pair[0]] + data[col_pair[1]]

    for col in relevant_features:
        self_interactions[f'{col}_squared'] = data[col] ** 2

    # Creating lags for all relevant features
    for col in relevant_features:
        lags[f'{col}_lag1'] = data[col].shift(1)
        lags[f'{col}_lag3'] = data[col].shift(3)

    # Concatenate all new features with the original data
    data = pd.concat([data, pd.DataFrame(interactions), pd.DataFrame(ratios),
                      pd.DataFrame(differences), pd.DataFrame(lags), pd.DataFrame(self_interactions), pd.DataFrame(additive)], axis=1)

    data['wind_magnitude'] = np.sqrt(data['wind_speed_u_10m:ms']**2 + data['wind_speed_v_10m:ms']**2)
    data['wind_direction'] = np.arctan2(data['wind_speed_v_10m:ms'], data['wind_speed_u_10m:ms'])
    data['solar_angle_impact'] = np.sin(np.radians(data['sun_elevation:d']))

    return data

# Location A

In [ ]:
train_a = pd.read_parquet('./data/A/train_targets.parquet')

X_train_estimated_a = pd.read_parquet('./data/A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('./data/A/X_train_observed.parquet')
X_test_estimated_a = pd.read_parquet('./data/A/X_test_estimated.parquet')

df = pd.concat([X_train_observed_a, X_train_estimated_a])

df = resample_to_hourly(df)
X_test_estimated_a = resample_to_hourly(X_test_estimated_a)

df = pd.merge(df, train_a, left_on='date_forecast', right_on='time', how='inner')
df = df.drop(columns=['snow_density:kgm3', 'snow_drift:idx', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'fresh_snow_12h:cm', 'snow_melt_10min:mm', 'elevation:m', 'prob_rime:p', 'dew_or_rime:idx', 'precip_5min:mm', 'rain_water:kgm2', 'snow_melt_10min:mm', 'wind_speed_w_1000hPa:ms'])
X_test_estimated_a = X_test_estimated_a.drop(columns=['snow_density:kgm3', 'snow_drift:idx', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'fresh_snow_12h:cm', 'snow_melt_10min:mm', 'elevation:m', 'prob_rime:p', 'dew_or_rime:idx', 'precip_5min:mm', 'rain_water:kgm2', 'snow_melt_10min:mm', 'wind_speed_w_1000hPa:ms'])

In [ ]:
cols_to_impute = ['ceiling_height_agl:m', 'cloud_base_agl:m']

imputer = IterativeImputer(max_iter=10, random_state=42)
X_test_estimated_a[cols_to_impute] = imputer.fit_transform(X_test_estimated_a[cols_to_impute])
df[cols_to_impute] = imputer.fit_transform(df[cols_to_impute])
df = df.dropna(subset=['pv_measurement'])

In [ ]:
df = is_estimated(df)
df = generate_solar_features_1(df)

X_test_estimated_a = is_estimated(X_test_estimated_a, 'date_forecast')
X_test_estimated_a = generate_solar_features_1(X_test_estimated_a)

In [ ]:
train_end_date = '2022-10-21'
df['time'] = pd.to_datetime(df['time'])

train_df = df[df['time'] < train_end_date]
remaining_data = df[df['time'] > train_end_date]

train_data, validation_df = train_test_split(remaining_data, test_size=0.5, random_state=42)
train_df = pd.concat([train_df, train_data], ignore_index=True)

# Identifying the features and the target variable
X_train = train_df.drop(columns=['pv_measurement', 'time', 'date_forecast'])
y_train = train_df['pv_measurement']
X_val = validation_df.drop(columns=['pv_measurement', 'time', 'date_forecast'])
y_val = validation_df['pv_measurement']

In [ ]:
# Combine training and validation data into a single dataset for AutoGluon
train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_val, y_val], axis=1)

# Specify the name of the target variable
label = 'pv_measurement'

# Create a TabularPredictor object
predictor = TabularPredictor(label=label, eval_metric="mean_absolute_error").fit(train_data=train_data, tuning_data=val_data, presets='medium_quality')

In [ ]:
results = predictor.fit_summary()

In [ ]:
feature_importance = predictor.feature_importance(val_data)

In [ ]:
best_features = feature_importance[feature_importance['importance'] > 0.1].index.tolist()

X_train = X_train[best_features]
train_data = pd.concat([X_train, y_train], axis=1)

X_val = X_val[best_features]
val_data = pd.concat([X_val, y_val], axis=1)

label = 'pv_measurement'

predictor = TabularPredictor(label=label, eval_metric="mean_absolute_error").fit(train_data=train_data, tuning_data=val_data, presets='medium_quality')

In [ ]:
results = predictor.fit_summary()

In [ ]:
X_test_estimated_a = X_test_estimated_a[best_features]

y_pred = predictor.predict(X_test_estimated_a)
y_pred = y_pred.clip(lower=0)
y_pred = y_pred.reset_index(drop=True)
y_pred.index.name = 'id'

In [ ]:
df = pd.DataFrame(y_pred)
df.to_csv('result_a.csv')

# Location B

In [ ]:
train_b = pd.read_parquet('./data/B/train_targets.parquet')

X_train_estimated_b = pd.read_parquet('./data/B/X_train_estimated.parquet')
X_train_observed_b = pd.read_parquet('./data/B/X_train_observed.parquet')
X_test_estimated_b = pd.read_parquet('./data/B/X_test_estimated.parquet')

df = pd.concat([X_train_observed_b, X_train_estimated_b])

df = resample_to_hourly(df)
X_test_estimated_b = resample_to_hourly(X_test_estimated_b)

df = pd.merge(df, train_b, left_on='date_forecast', right_on='time', how='inner')
df = df.drop(columns=['snow_density:kgm3', 'elevation:m', 'snow_drift:idx', 'snow_melt_10min:mm', 'wind_speed_w_1000hPa:ms', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'fresh_snow_12h:cm', 'precip_5min:mm', 'rain_water:kgm2', 'snow_drift:idx', 'snow_melt_10min:mm', 'wind_speed_w_1000hPa:ms'])
X_test_estimated_b = X_test_estimated_b.drop(columns=['snow_density:kgm3', 'elevation:m', 'snow_drift:idx', 'snow_melt_10min:mm', 'wind_speed_w_1000hPa:ms', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'fresh_snow_12h:cm', 'precip_5min:mm', 'rain_water:kgm2', 'snow_drift:idx', 'snow_melt_10min:mm', 'wind_speed_w_1000hPa:ms'])

In [ ]:
cols_to_impute = ['ceiling_height_agl:m', 'cloud_base_agl:m']

imputer = IterativeImputer(max_iter=10, random_state=42)
X_test_estimated_b[cols_to_impute] = imputer.fit_transform(X_test_estimated_b[cols_to_impute])
df[cols_to_impute] = imputer.fit_transform(df[cols_to_impute])
df = df.dropna(subset=['pv_measurement'])

In [ ]:
segments = find_long_constant_periods(train_b['pv_measurement'], threshold=5)
df = remove_constant_periods(df, segments)
df = remove_unwanted_rows(df)
df = lag_features_by_one_hour(df, ['diffuse_rad_1h:J', 'direct_rad_1h:J', 'clear_sky_energy_1h:J'])
df = is_estimated(df)

X_test_estimated_b = is_estimated(X_test_estimated_b, 'date_forecast')
X_test_estimated_b = lag_features_by_one_hour(X_test_estimated_b, ['diffuse_rad_1h:J', 'direct_rad_1h:J', 'clear_sky_energy_1h:J'], 'date_forecast')

In [ ]:
train_end_date = '2022-10-21'
df['time'] = pd.to_datetime(df['time'])

train_df = df[df['time'] < train_end_date]
remaining_data = df[df['time'] > train_end_date]

train_data, validation_df = train_test_split(remaining_data, test_size=0.5, random_state=42)
train_df = pd.concat([train_df, train_data], ignore_index=True)

# Identifying the features and the target variable
X_train = train_df.drop(columns=['pv_measurement', 'time', 'date_forecast'])
y_train = train_df['pv_measurement']
X_val = validation_df.drop(columns=['pv_measurement', 'time', 'date_forecast'])
y_val = validation_df['pv_measurement']

In [ ]:
# Combine training and validation data into a single dataset for AutoGluon
train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_val, y_val], axis=1)

# Specify the name of the target variable
label = 'pv_measurement'

# Create a TabularPredictor object
predictor = TabularPredictor(label=label, eval_metric="mean_absolute_error").fit(train_data=train_data, tuning_data=val_data, presets='best_quality', num_gpus=1, num_stack_levels=0, use_bag_holdout=True)

In [ ]:
results = predictor.fit_summary()

In [ ]:
feature_importance = predictor.feature_importance(val_data)

In [ ]:
y_pred = predictor.predict(X_test_estimated_b)
y_pred = y_pred.clip(lower=0)
y_pred = y_pred.reset_index(drop=True)
y_pred.index.name = 'id'

In [ ]:
df = pd.DataFrame(y_pred)
df.to_csv('result_b.csv')

# Location C

In [ ]:
train_c = pd.read_parquet('./data/C/train_targets.parquet')

X_train_estimated_c = pd.read_parquet('./data/C/X_train_estimated.parquet')
X_train_observed_c = pd.read_parquet('./data/C/X_train_observed.parquet')
X_test_estimated_c = pd.read_parquet('./data/C/X_test_estimated.parquet')

df = pd.concat([X_train_observed_c, X_train_estimated_c])

df = resample_to_hourly(df)
X_test_estimated_c = resample_to_hourly(X_test_estimated_c)

df = pd.merge(df, train_c, left_on='date_forecast', right_on='time', how='inner')
df = df.drop(columns=['snow_density:kgm3', 'elevation:m', 'snow_drift:idx', 'snow_melt_10min:mm', 'wind_speed_w_1000hPa:ms', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'fresh_snow_12h:cm', 'precip_5min:mm', 'rain_water:kgm2', 'snow_drift:idx', 'snow_melt_10min:mm', 'wind_speed_w_1000hPa:ms'])
X_test_estimated_c = X_test_estimated_c.drop(columns=['snow_density:kgm3', 'elevation:m', 'snow_drift:idx', 'snow_melt_10min:mm', 'wind_speed_w_1000hPa:ms', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'fresh_snow_12h:cm', 'precip_5min:mm', 'rain_water:kgm2', 'snow_drift:idx', 'snow_melt_10min:mm', 'wind_speed_w_1000hPa:ms'])

In [ ]:
cols_to_impute = ['ceiling_height_agl:m', 'cloud_base_agl:m']

imputer = IterativeImputer(max_iter=10, random_state=42)
X_test_estimated_c[cols_to_impute] = imputer.fit_transform(X_test_estimated_c[cols_to_impute])
df[cols_to_impute] = imputer.fit_transform(df[cols_to_impute])
df = df.dropna(subset=['pv_measurement'])

In [ ]:
segments = find_long_constant_periods(train_c['pv_measurement'], threshold=5)
df = remove_constant_periods(df, segments)
df = is_estimated(df)
df = lag_features_by_one_hour(df, ['diffuse_rad_1h:J', 'direct_rad_1h:J', 'clear_sky_energy_1h:J'])

X_test_estimated_c = is_estimated(X_test_estimated_c, 'date_forecast')
X_test_estimated_c = lag_features_by_one_hour(X_test_estimated_c, ['diffuse_rad_1h:J', 'direct_rad_1h:J', 'clear_sky_energy_1h:J'], 'date_forecast')

In [ ]:
train_end_date = '2022-10-21'
df['time'] = pd.to_datetime(df['time'])

train_df = df[df['time'] < train_end_date]
remaining_data = df[df['time'] > train_end_date]

train_data, validation_df = train_test_split(remaining_data, test_size=0.5, random_state=42)
train_df = pd.concat([train_df, train_data], ignore_index=True)

# Identifying the features and the target variable
X_train = train_df.drop(columns=['pv_measurement', 'time', 'date_forecast'])
y_train = train_df['pv_measurement']
X_val = validation_df.drop(columns=['pv_measurement', 'time', 'date_forecast'])
y_val = validation_df['pv_measurement']

In [ ]:
# Combine training and validation data into a single dataset for AutoGluon
train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_val, y_val], axis=1)

# Specify the name of the target variable
label = 'pv_measurement'

# Create a TabularPredictor object
predictor = TabularPredictor(label=label, eval_metric="mean_absolute_error").fit(train_data=train_data, tuning_data=val_data, presets='best_quality', num_gpus=1, num_stack_levels=0, use_bag_holdout=True)

In [ ]:
results = predictor.fit_summary()

In [ ]:
y_pred = predictor.predict(X_test_estimated_c)
y_pred = y_pred.clip(lower=0)
y_pred = y_pred.reset_index(drop=True)
y_pred.index.name = 'id'

In [ ]:
df = pd.DataFrame(y_pred)
df.to_csv('result_c.csv')